In [1]:
# Libraries
import os
import json
import pandas as pd
import csv

In [2]:
from explanation_strategy_evaluation_metrics import ExplanationStrategy
from explanation_strategy_evaluation_metrics import timeliness
from explanation_strategy_evaluation_metrics import popularity
from explanation_strategy_evaluation_metrics import completeness
from explanation_strategy_evaluation_metrics import diversity
from explanation_strategy_evaluation_metrics import serendipity
from explanation_strategy_evaluation_metrics import granularity

In [3]:
# Specify the directory path
directory = ".\Trees_Random"
list_use_cases_global = list() # global variable 

### Code to translate from json to graph

In [4]:
intents = {}
intents["DEBUGGING"] = ["Is this the same outcome for similar instances?", "Is this instance a common occurrence?"]
intents["TRANSPARENCY"] = ["What is the impact of feature X on the outcome?","How does feature X impact the outcome?","What are the necessary features that guarantee this outcome?","Why does the AI system have given outcome A?","Which feature contributed to the current outcome?","How does the AI system respond to feature X?","What is the goal of the AI system?","What is the scope of the AI system capabilities?","What features does the AI system consider?","What are the important features for the AI system?", "What is the impact of feature X on the AI system?","How much evidence has been considered to build the AI system?", "How much evidence has been considered in the current outcome?","What are the possible outcomes of the AI system?","What features are used by the AI system?"] 
intents["PERFORMANCE"] = ["How confident is the AI system with the outcome?","Which instances get a similar outcome?","Which instances get outcome A?","What are the results when others use the AI System?","How accurate is the AI system?","How reliable is the AI system?","In what situations does the AI system make errors?","What are the limitations of the AI system?","In what situations is the AI system likely to be correct?"] 
intents["COMPLIANCY"] = ["How well does the AI system capture the real-world?","Why are instances A and B given different outcomes?"]
intents["COMPREHENSIBILITY"] = ["How to improve the AI system performance?","What does term X mean?","What is the overall logic of the AI system?","What kind of algorithm is used in the AI system?"]
intents["EFFECTIVENESS"] = ["What would be the outcome if features X is changed to value V?","What other instances would get the same outcome?","How does the AI system react if feature X is changed?","What is the impact of the current outcome?"] 
intents["ACTIONABILITY"] = ["What are the alternative scenarios available?","What type of instances would get a different outcome?","How can I change feature X to get the same outcome?","How to get a different outcome?","How to change the instance to get a different outcome?","How to change the instance to get outcome {outcome}?","Why does the AI system have given outcome A not B?","Which features need changed to get a different outcome?"] 


In [5]:
def typeQuestion(question):
    question_type = [key for key in intents.keys() if question in intents[key]]
    if question_type == []: 
        print("That question (" + question + ") is not in our catalog")
        return "NO_QUESTION"
    else:
        return question_type[0]

In [6]:
# Author of this cell: Preeja Pradeep (Github: https://github.com/preejapradeep)
# Code to tranform BTs in json format to graph format
# Edited from original source (here: https://github.com/preejapradeep/Constructive-Reuse/blob/main/json2graph.py)

def print_node_instances(node_id, nodes_dict, node_list, id_list): 
    node = nodes_dict[node_id]
    
    node_instance = node['Instance']
    if node_instance is None:
        return None
    elif node_instance == "User Question":
        node_instance = node["params"]["Question"]["value"]
    node_list.append(node_instance)
    id_list.append(node_id)

    if 'firstChild' in node:
        first_child_id = node['firstChild']['Id']
        print_node_instances(first_child_id, nodes_dict, node_list, id_list)
        next_child = node['firstChild'].get('Next')

        while next_child is not None:
            next_child_id = next_child['Id']
            print_node_instances(next_child_id, nodes_dict, node_list, id_list)
            next_child = next_child.get('Next')

    return node_list, id_list


def get_index(node_id, nodes_dict, id_list):
    node = nodes_dict[node_id]
    node_instance = node.get('Instance')
    node_index = id_list.index(node_id)
    node_index = node_index + 1

    return node_index, node_instance


def find_parent(node_id, node, parent_child_dict, id_list, nodes_dict):
    parent_index, parent_instance = get_index(node_id, nodes_dict, id_list)
    
    if 'firstChild' in node:
        first_child_id = node['firstChild']['Id']
        child_index, child_instance = get_index(first_child_id, nodes_dict, id_list)

        if parent_index not in parent_child_dict:
            parent_child_dict[parent_index] = []
        if child_index not in parent_child_dict[parent_index]:
            parent_child_dict[parent_index].append(child_index)
        
        next_child = node['firstChild'].get('Next')
        while next_child is not None:
            next_child_id = next_child['Id']
            child_index, child_instance = get_index(next_child_id, nodes_dict, id_list)
            if child_index not in parent_child_dict[parent_index]:
                parent_child_dict[parent_index].append(child_index)  # Add child index to the parent's list
            next_child = next_child.get('Next')

        return parent_instance
    
    
def create_parent_child_dict(nodes_dict, node_list, id_list): 
    parent_child_dict = {}   
    # root = node_list[0] #r 
    parent_child_dict[0] = [1]  # Add root node with index 0

    for i, (instance, node_id) in enumerate(zip(node_list[1:], id_list), start=1):
        node_index = i
        node_id =id_list[node_index-1]
        node = nodes_dict[node_id]
        find_parent(node_id, node, parent_child_dict, id_list, nodes_dict)
    
    return parent_child_dict


def build_adjacency_list(node_list, parent_child_dict): 
    adjacency_list = [[] for _ in range(len(node_list))]

    for node_index, node_instance in enumerate(node_list):
        if node_index in parent_child_dict:
            children = parent_child_dict[node_index]
            adjacency_list[node_index] = children

    return adjacency_list


def translateCasesFromJSONtoGraph(case):
    
    tree_dict, nodes_dict, parent_child_dict = {},{},{}
    node_list = ['r'] # Added 'r' as the default root node in the node list
    id_list =[] #List of node id's 

    tree = case
    #for idx, obj in enumerate(case, start=1):
        #print(idx)
        #print(obj)
        #trees = obj['data']['trees']
        
        # Get the 'nodes' from 'trees'
        #for tree in trees:
    nodes = tree.get('nodes', {})
    nodes_dict.update(nodes)
    # Get the root node
    root_node_id = tree.get('root')    

    # Call the recursive function to print node instances
    node_list, id_list= print_node_instances(root_node_id, nodes_dict, node_list = ['r'], id_list =[])

    # Call the function to create the parent_child dictionary
    parent_child_dict = create_parent_child_dict(nodes_dict, node_list, id_list)
    # Build the adjacency list from the behavior tree
    adjacency_list = build_adjacency_list(node_list, parent_child_dict)

    #tree_key = f'tree_{idx}'
    #   tree_dict[tree_key] = trees
    #tree_dict[tree_key] = {
          #'tree_json': trees,
    tree_dict = {
      'nodes': node_list,
      'adj': adjacency_list
    }

    return tree_dict

### Load the explainer catalog

In [7]:
with open("local_explainer_catalog.csv", 'r') as file:
    csv_reader = csv.DictReader(file)
    data = [row for row in csv_reader]
# print(data)

In [8]:
explainers_knowledge = {e["Explainer_name"]: {'ExplainabilityTechnique': e['ExplainabilityTechnique'], 'Dataset Type': e['Dataset Type'], 'ExplanationType': e['ExplanationType'], 'Concurrentness':e['Concurrentness'], 'Portability':e['Portability'], 'Scope':e['Scope'], 'Target':e['Target'], 'InformationContentEntity':e['InformationContentEntity'], 'computational_complexity':e['computational_complexity'], 'AI_Method':e['AI_Method'], 'AI_Task':e['AI_Task'], 'Implementation_Framework':e['Implementation_Framework'], 'Model_Access':e['Model_Access'], 'Data_required':e['Data_required'], 'popularity':int(e['popularity'])} for e in data}

In [9]:
# explainers_knowledge

### Read the BT json and translate them into graphs

In [10]:
def includeResult(path):
    """
        Auxiliar function to get the results for a random tree
    """
    result = ""
    if os.path.exists(path) == True:
        f = open(path, "r")
        translation = translateCasesFromJSONtoGraph(json.loads(f.read()))
        result = translation
    else:
        result = "NaN"
        
    return result

In [11]:
# Open the directory and view its contents
contents = os.listdir(directory)

# lists of lists of dictionaries: each dict: for each use case; each dictionary for a random tree
list_graphs = list() 
# Print the contents of the directory
for use_case in contents:
    use_case_list = dict()
    list_use_cases_global.append(directory + "\\" + use_case + "\\Explainer aplicable.json")
    # open the 10 random trees
    # if exists: 
    # open and translate original bt, explainer reuse, BT reuse, and explainer + BT reuse 
    for i in range(0,10):
        random_tree = dict()
        directory_original = directory + "\\" + use_case + "\\" + str(i) + ".json"
        directory_reuse_explainer = directory + "\\" + use_case + "\\" + str(i) + " reuse explaner.json"
        directory_reuse_bt = directory + "\\" + use_case + "\\" + str(i) + " reuse tree.json"
        directory_reuse_bt_explainer = directory + "\\" + use_case + "\\" + str(i) + " reuse tree and explaner.json"
        
        random_tree[str(i) + "_original"] = includeResult(directory_original)
        random_tree[str(i) + "_reuse_explainer"] = includeResult(directory_reuse_explainer)
        random_tree[str(i) + "_reuse_bt"] = includeResult(directory_reuse_bt)        
        random_tree[str(i) + "_reuse_bt_explainer"] = includeResult(directory_reuse_bt_explainer)
        
        use_case_list[use_case + "_random_tree_" + str(i)] = random_tree
        
            
    list_graphs.append(use_case_list) 
    

In [12]:
# print(list_graphs)

### Create table of results of each use case

In [13]:
def get_explanation_strategy_from_BT(graph_tree, explainers_knowledge):
    """
        It gets an ExplanationStrategy object from a BT in graph format
    """
    
    # we need to get the explainers in the strategy
    explainers = [e for e in graph_tree['nodes'] if e[0] == "/"]
    
    # Get the explainer properties
    exp_dict = {key: value for key, value in explainers_knowledge.items() if key in explainers}
    
    # build the object
    explanation_strategy_object = ExplanationStrategy(exp_dict,structure_input=graph_tree)
    
    return explanation_strategy_object 
    

In [14]:
def create_strategy(graph, explainers_knowledge):
    my_object = get_explanation_strategy_from_BT(graph, explainers_knowledge)
    return my_object

In [15]:
def calculateMetrics(my_object):
    """
        Calculates the evaluation metrics
    """
    return timeliness(my_object), popularity(my_object, 1, 3), completeness(my_object), diversity(my_object), serendipity(my_object), granularity(my_object), my_object.numberNodes(), my_object.numberExplainers()

In [16]:
def getKey(original_string):
    my_key = ""
    if "_original" in original_string:
        my_key = "Original_BT"
    elif "_reuse_explainer" in original_string:
        my_key = "Explainer_Reuse_Result"
    elif "_reuse_bt_explainer" in original_string:
        my_key = "Explainer_BT_Reuse_Result"
    elif "_reuse_bt" in original_string:
        my_key = "BT_Reuse_Result"
    
    return my_key

In [17]:
def number_applicable_explainers(explainers_use_case, explainers_in_tree):
    return len([x for x in explainers_in_tree if x in explainers_use_case])

In [18]:
def applicable_explainers(path):
    f = open(path, "r")
    json_text = json.loads(f.read())
    
    a_e = [e for e in list(json_text.keys()) if json_text[e]["flag"] == True]
    return a_e

In [19]:
def create_results(results, explainers_knowledge):
    """
        Create the table of results
    """
    cont_use_case = 0
    result_list = []
    for use_case in results:
        cont_random_tree = 0
        
        # getting the path to check the applicable explainers
        my_use_case = list(use_case.keys())[0].replace("_random_tree_0","")
        path = [x for x in list_use_cases_global if my_use_case in x][0]
        _exp = applicable_explainers(path)
        
        for _,random_tree_value in use_case.items():

            result_use_case = dict()
            result_use_case["Use_Case"] = cont_use_case
            result_use_case["Random_Tree"] = cont_random_tree
            for tree, value in random_tree_value.items():
                
                if value != "NaN":
                    my_object = create_strategy(value, explainers_knowledge)
                    t, p, c, d, s, g, n, e = calculateMetrics(my_object)
                    app_exp = number_applicable_explainers(_exp,my_object.getExplainersNames())

                else:
                    t, p, c, d, s, g, n, e, app_exp = "NaN","NaN","NaN","NaN","NaN","NaN","NaN","NaN","NaN"

                result_use_case[getKey(tree) + "_Timeliness"] = t
                result_use_case[getKey(tree) + "_Popularity"] = p
                result_use_case[getKey(tree) + "_Completeness"] = c
                result_use_case[getKey(tree) + "_Diversity"] = d
                result_use_case[getKey(tree) + "_Serendipity"] = s
                result_use_case[getKey(tree) + "_Granularity"] = g
                result_use_case[getKey(tree) + "_#_Nodes"] = n
                result_use_case[getKey(tree) + "_#_Explainers"] = e
                result_use_case[getKey(tree) + "_#_Applicable_Explainers"] = app_exp
                
            result_list.append(result_use_case)
            cont_random_tree = cont_random_tree + 1
        
        cont_use_case = cont_use_case + 1
    return result_list

In [20]:
result_series = create_results(list_graphs, explainers_knowledge)

In [21]:
df = pd.DataFrame(result_series)

In [22]:
df

,Use_Case,Random_Tree,Original_BT_Timeliness,Original_BT_Popularity,Original_BT_Completeness,Original_BT_Diversity,Original_BT_Serendipity,Original_BT_Granularity,Original_BT_#_Nodes,Original_BT_#_Explainers,...,BT_Reuse_Result_#_Applicable_Explainers,Explainer_BT_Reuse_Result_Timeliness,Explainer_BT_Reuse_Result_Popularity,Explainer_BT_Reuse_Result_Completeness,Explainer_BT_Reuse_Result_Diversity,Explainer_BT_Reuse_Result_Serendipity,Explainer_BT_Reuse_Result_Granularity,Explainer_BT_Reuse_Result_#_Nodes,Explainer_BT_Reuse_Result_#_Explainers,Explainer_BT_Reuse_Result_#_Applicable_Explainers
0,0,0,0.571429,0.500000,0.433333,0.566667,"(False, No serendipia)",0.425000,8,3,...,2,0.857143,0.75,0.5,0.5,"(False, No serendipia)",0.416667,6,2,2
1,0,1,0.857143,0.700000,0.633333,0.366667,"(False, No serendipia)",0.350000,12,5,...,2,0.857143,0.75,0.5,0.5,"(False, No serendipia)",0.416667,6,2,2
2,0,2,0.571429,0.500000,0.650000,0.350000,"(False, No serendipia)",0.350000,12,5,...,2,0.857143,0.75,0.5,0.5,"(False, No serendipia)",0.416667,6,2,2
3,0,3,0.571429,0.000000,0.400000,0.600000,"(False, No serendipia)",0.333333,6,2,...,1,0.285714,0.50,1.0,0.0,"(False, No serendipia)",0.400000,4,1,1
4,0,4,0.857143,0.333333,0.566667,0.433333,"(False, No serendipia)",0.300000,8,3,...,1,0.285714,0.50,1.0,0.0,"(False, No serendipia)",0.400000,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,14,5,0.571429,0.642857,0.722222,0.277778,"(False, No serendipia)",0.375000,16,7,...,1,0.285714,0.50,1.0,0.0,"(False, No serendipia)",0.400000,4,1,1
146,14,6,0.571429,0.625000,0.644444,0.355556,"(False, No serendipia)",0.380000,10,4,...,1,0.285714,0.50,1.0,0.0,"(False, No serendipia)",0.400000,4,1,1
147,14,7,0.857143,0.500000,0.633333,0.366667,"(False, No serendipia)",0.350000,12,5,...,1,0.285714,0.50,1.0,0.0,"(False, No serendipia)",0.400000,4,1,1
148,14,8,0.857143,0.500000,0.750000,0.250000,"(False, No serendipia)",0.390000,20,9,...,1,0.571429,0.00,1.0,0.0,"(False, No serendipia)",0.400000,4,1,1


In [23]:
df[["Original_BT_#_Explainers","Original_BT_#_Applicable_Explainers","Explainer_Reuse_Result_#_Explainers","Explainer_Reuse_Result_#_Applicable_Explainers","BT_Reuse_Result_#_Explainers","BT_Reuse_Result_#_Applicable_Explainers","Explainer_BT_Reuse_Result_#_Explainers","Explainer_BT_Reuse_Result_#_Applicable_Explainers"]] 

,Original_BT_#_Explainers,Original_BT_#_Applicable_Explainers,Explainer_Reuse_Result_#_Explainers,Explainer_Reuse_Result_#_Applicable_Explainers,BT_Reuse_Result_#_Explainers,BT_Reuse_Result_#_Applicable_Explainers,Explainer_BT_Reuse_Result_#_Explainers,Explainer_BT_Reuse_Result_#_Applicable_Explainers
0,3,1,3,3,2,2,2,2
1,5,2,5,5,2,2,2,2
2,5,3,4,4,2,2,2,2
3,2,0,2,2,1,1,1,1
4,3,2,3,3,1,1,1,1
...,...,...,...,...,...,...,...,...
145,7,3,7,7,1,1,1,1
146,4,0,4,4,1,1,1,1
147,5,2,5,5,1,1,1,1
148,9,5,8,8,1,1,1,1


When we have that the explainer reuse has less amount of explainers than the original one is because some explainers in the reuse of explainers might be repeated

In [24]:
df.to_csv("results.csv")